In [1]:
import prbench
import numpy as np
import time

prbench.register_all_environments()
env = prbench.make(
    "prbench/TidyBot3D-ground-o5-v0",
    render_images=True,
    show_viewer=False,
    show_images=True
)

obs, info = env.reset(seed=0)
robot = env.env.env._tidybot_robot_env
sim = robot.sim

print("After reset - arm qpos[38:45]:", sim.data.qpos[38:45])
print("After reset - arm ctrl[3:10]:", sim.data.ctrl[3:10])
print("Expected retract position:", [0.0, -0.34906585, 3.14159265, -2.54818071, 0.0, -0.87266463, 1.57079633])
print("Current base position:", env.env.env._tidybot_robot_env.qpos_base)
print()

# Test a few steps to see if warning persists
imgs = []
for i_step in range(100):
    img = env.env.env._tidybot_robot_env._get_camera_images()["overview_image"]
    imgs.append(img)

    # action = env.action_space.sample()
    action = np.array([0, 0, 0, 0, 0, 0.5, 1, 0, 0, 0, 0.0])
    print(f"Step {i_step} - action:", action)
    obs, reward, terminated, truncated, info = env.step(action)


    # obs, info = env.reset(seed=i_step)
    # sim = env.env.env._tidybot_robot_env.sim


    print(f"Step {i_step} - arm qpos[38:45]:", robot.qpos_arm)
    print(f"Step {i_step} - base qpos[35:38]:", robot.qpos_base)
    print("")

# env.close()

/home/yixuan/miniconda3/envs/pr_env/lib/python3.10/site-packages/gymnasium/spaces/box.py:236: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/home/yixuan/miniconda3/envs/pr_env/lib/python3.10/site-packages/gymnasium/spaces/box.py:306: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


After reset - arm qpos[38:45]: [ 0.         -0.34906585  3.14159265 -2.54818071  0.         -0.87266463
  1.57079633]
After reset - arm ctrl[3:10]: [ 0.         -0.34906585  3.14159265 -2.54818071  0.         -0.87266463
  1.57079633]
Expected retract position: [0.0, -0.34906585, 3.14159265, -2.54818071, 0.0, -0.87266463, 1.57079633]
Current base position: [ 0.27392337 -0.46042657 -2.88414841]

Step 0 - action: [0.  0.  0.  0.  0.  0.5 1.  0.  0.  0.  0. ]
Step 0 - arm qpos[38:45]: [ 1.62267508e-03  2.76998579e-02  3.14045082e+00 -1.08204773e+00
  6.23801130e-04 -2.74248134e-01  1.56925820e+00]
Step 0 - base qpos[35:38]: [ 0.27383334 -0.46024189 -2.88259127]

Step 1 - action: [0.  0.  0.  0.  0.  0.5 1.  0.  0.  0.  0. ]
Step 1 - arm qpos[38:45]: [ 8.78855496e-03  2.37501318e-01  3.14359259e+00  1.88076088e-01
 -6.37899936e-05  7.01943885e-03  1.56078472e+00]
Step 1 - base qpos[35:38]: [ 0.27240482 -0.45943195 -2.87578041]

Step 2 - action: [0.  0.  0.  0.  0.  0.5 1.  0.  0.  0.  0. ]

In [ ]:
import mujoco

mujoco.viewer.launch(
    sim.model._model,  # pylint: disable=protected-access
    sim.data,
    show_left_ui=False,
    show_right_ui=False,
)


In [2]:
import imageio
imageio.mimsave("tmp_test_tidybot_robot_env.mp4", imgs, fps=100)  # Save the video if needed

In [3]:
base_qpos_indices = [
    sim.model.get_joint_qpos_addr(name) for name in ["joint_x", "joint_y", "joint_th"]
]
print(base_qpos_indices)


[np.int32(0), np.int32(1), np.int32(2)]


In [4]:
base_qvel_indices = [
    sim.model.get_joint_qvel_addr(name) for name in ["joint_x", "joint_y", "joint_th"]
]
print(base_qvel_indices)


[np.int32(0), np.int32(1), np.int32(2)]


In [5]:
arm_ctrl_indices = [sim.model._actuator_name2id[jnt] for jnt in ["joint_1", "joint_2", "joint_3", "joint_4", "joint_5", "joint_6", "joint_7"]]
print(arm_ctrl_indices)

[3, 4, 5, 6, 7, 8, 9]


In [6]:
sim.model.actuator_names

('joint_x',
 'joint_y',
 'joint_th',
 'joint_1',
 'joint_2',
 'joint_3',
 'joint_4',
 'joint_5',
 'joint_6',
 'joint_7',
 'fingers_actuator')

In [7]:
robot = env.env.env._tidybot_robot_env
robot.ctrl_arm

array([ 0.        , -0.34906585,  3.14159265, -2.54818071,  0.        ,
       -0.87266463,  1.57079633])

In [5]:
xml_str = sim.model._model.get_xml()
print(xml_str[:500])  # Print the first 500 characters for preview

AttributeError: 'mujoco._structs.MjModel' object has no attribute 'get_xml'

In [4]:
# Debug joint indices and DOF structure
env = prbench.make("prbench/TidyBot3D-ground-o5-v0", render_images=False, show_viewer=False)
obs, info = env.reset(seed=123)
sim = env.env.env._tidybot_robot_env.sim

print("Total qpos length:", len(sim.data.qpos))
print("Total qvel length:", len(sim.data.qvel))
print("Total ctrl length:", len(sim.data.ctrl))

# Check joint mappings
base_joints = ["joint_x", "joint_y", "joint_th"]
arm_joints = ["joint_1", "joint_2", "joint_3", "joint_4", "joint_5", "joint_6", "joint_7"]

print("\nBase joint IDs:")
for name in base_joints:
    joint_id = sim.model._joint_name2id[name]
    print(f"  {name}: joint_id={joint_id}")
    
print("\nArm joint IDs:")
for name in arm_joints:
    joint_id = sim.model._joint_name2id[name]
    print(f"  {name}: joint_id={joint_id}")
    
print("\nBase actuator IDs:")
for name in base_joints:
    actuator_id = sim.model._actuator_name2id[name]
    print(f"  {name}: actuator_id={actuator_id}")
    
print("\nArm actuator IDs:")
for name in arm_joints:
    actuator_id = sim.model._actuator_name2id[name]
    print(f"  {name}: actuator_id={actuator_id}")

# Check current qpos values
print("\nCurrent qpos:", sim.data.qpos)
print("Current qvel:", sim.data.qvel)
print("Current ctrl:", sim.data.ctrl)

env.close()

Total qpos length: 53
Total qvel length: 48
Total ctrl length: 11

Base joint IDs:
  joint_x: joint_id=5
  joint_y: joint_id=6
  joint_th: joint_id=7

Arm joint IDs:
  joint_1: joint_id=8
  joint_2: joint_id=9
  joint_3: joint_id=10
  joint_4: joint_id=11
  joint_5: joint_id=12
  joint_6: joint_id=13
  joint_7: joint_id=14

Base actuator IDs:
  joint_x: actuator_id=0
  joint_y: actuator_id=1
  joint_th: actuator_id=2

Arm actuator IDs:
  joint_1: actuator_id=3
  joint_2: actuator_id=4
  joint_3: actuator_id=5
  joint_4: actuator_id=6
  joint_5: actuator_id=7
  joint_6: actuator_id=8
  joint_7: actuator_id=9

Current qpos: [ 0.          0.          0.          1.          0.          0.
  0.          0.          0.          0.          1.          0.
  0.          0.          0.          0.          0.          1.
  0.          0.          0.          0.          0.          0.
  1.          0.          0.          0.          0.          0.
  0.          1.          0.          0.     

In [2]:
obs.shape

(2764901,)

In [ ]:
env.env.env._tidybot_robot_env.get_obs().keys()

dict_keys(['qpos', 'qvel', 'overview_image'])

In [2]:
action

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
env.env.env._tidybot_robot_env.camera_names

['overview']

In [3]:
env.env.env._tidybot_robot_env.render() # TODO

AttributeError: 'TidyBotRobotEnv' object has no attribute 'render'

In [5]:
env.env.env._tidybot_robot_env._get_camera_images().keys()

dict_keys(['overview_image', 'base_image', 'wrist_image'])

In [2]:
env.close()

In [3]:
# Debug: Analyze DOF structure and controller state
import prbench
import numpy as np

prbench.register_all_environments()
env = prbench.make(
    "prbench/TidyBot3D-ground-o5-v0",
    render_images=False,  # Disable rendering for debugging
    show_viewer=False,
    show_images=False
)

# Reset and examine the model structure
obs, info = env.reset(seed=123)
sim = env.env.env._tidybot_robot_env.sim

print("Total DOFs:", sim.model._model.nq)
print("Total joints:", sim.model._model.njnt)
print("Total actuators:", sim.model._model.nu)

# Check joint names around DOF 32
for i in range(max(0, 30), min(sim.model._model.njnt, 35)):
    joint_name = sim.model._model.joint_id2name(i)
    print(f"Joint {i}: {joint_name}")

# Check qpos and ctrl values
print("\nqpos shape:", sim.data.qpos.shape)
print("ctrl shape:", sim.data.ctrl.shape)
print("qpos around DOF 32:", sim.data.qpos[30:35])
print("ctrl around DOF 32:", sim.data.ctrl[30:35] if len(sim.data.ctrl) > 30 else "Not enough actuators")

# Check arm controller state
arm_ctrl = env.env.env._tidybot_robot_env.arm_controller
if arm_ctrl:
    print("\nArm controller qpos:", arm_ctrl.qpos)
    print("Arm controller ctrl:", arm_ctrl.ctrl)
    print("Retract qpos:", arm_ctrl.retract_qpos)

env.close()

/opt/anaconda3/envs/prbench/lib/python3.10/site-packages/gymnasium/envs/registration.py:636: UserWarning: WARN: Overriding environment prbench/TidyBot3D-ground-o3-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/opt/anaconda3/envs/prbench/lib/python3.10/site-packages/gymnasium/envs/registration.py:636: UserWarning: WARN: Overriding environment prbench/TidyBot3D-ground-o5-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/opt/anaconda3/envs/prbench/lib/python3.10/site-packages/gymnasium/envs/registration.py:636: UserWarning: WARN: Overriding environment prbench/TidyBot3D-ground-o7-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/opt/anaconda3/envs/prbench/lib/python3.10/site-packages/gymnasium/envs/registration.py:636: UserWarning: WARN: Overriding environment prbench/TidyBot3D-table-o3-v0 already in registry.
  logger.warn(f"Overridin



Total DOFs: 53
Total joints: 23
Total actuators: 11

qpos shape: (53,)
ctrl shape: (11,)
qpos around DOF 32: [-1.17386117e-04  1.00000000e+00  9.43342071e-17  4.43125536e-17
 -1.88728640e-17]
ctrl around DOF 32: Not enough actuators

Arm controller qpos: [ 1.00000000e+00 -8.53713322e-16  8.59210925e-16 -9.06030218e-17
  7.95454636e-17  4.28346651e-18 -1.17386117e-04]
Arm controller ctrl: [0. 0. 0. 0. 0. 0. 0.]
Retract qpos: [ 0.         -0.34906585  3.14159265 -2.54818071  0.         -0.87266463
  1.57079633]


In [4]:
# Test the fix
import prbench
import numpy as np

prbench.register_all_environments()
env = prbench.make(
    "prbench/TidyBot3D-ground-o5-v0",
    render_images=False,
    show_viewer=False,
    show_images=False
)

# Reset and check if instability warning appears
print("Resetting environment...")
obs, info = env.reset(seed=123)
sim = env.env.env._tidybot_robot_env.sim

# Check arm controller state after reset
arm_ctrl = env.env.env._tidybot_robot_env.arm_controller
print("\nAfter reset:")
print("Arm controller qpos:", arm_ctrl.qpos)
print("Arm controller ctrl:", arm_ctrl.ctrl)
print("Retract qpos:", arm_ctrl.retract_qpos)
print("Are they close?", np.allclose(arm_ctrl.qpos, arm_ctrl.retract_qpos, atol=1e-3))

# Try a few steps
print("\nTaking simulation steps...")
for i in range(5):
    action = env.action_space.sample() * 0.1  # Small actions
    obs, reward, terminated, truncated, info = env.step(action)
    print(f"Step {i+1} completed successfully")

print("\nTest completed - no instability warnings!")
env.close()

Resetting environment...

After reset:
Arm controller qpos: [ 0.         -0.34906585  3.14159265 -2.54818071  0.         -0.87266463
  1.57079633]
Arm controller ctrl: [ 0.         -0.34906585  3.14159265 -2.54818071  0.         -0.87266463
  1.57079633]
Retract qpos: [ 0.         -0.34906585  3.14159265 -2.54818071  0.         -0.87266463
  1.57079633]
Are they close? True

Taking simulation steps...
[DEBUG] otg_int.current_position [0.0, -0.34906585, 3.14159265, -2.54818071, 0.0, -0.87266463, 1.57079633]
[DEBUG] otg_out.new_position [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[DEBUG] otg_int.current_position [-8.377580409572781e-06, -0.34905747241959045, 3.141584272419591, -2.5481723324195906, 1.5707963267948967e-05, -0.872680337963268, 1.5707806220367322]
[DEBUG] otg_out.new_position [-8.377580409572781e-06, -0.34905747241959045, 3.141584272419591, -2.5481723324195906, 1.5707963267948967e-05, -0.872680337963268, 1.5707806220367322]
[DEBUG] otg_int.current_position [-3.3510321638291124e-05, 

ValueError: -101 is not a valid Result.

In [6]:
# Clean restart to test the Ruckig fix
import sys
import importlib

# Clear any cached modules
modules_to_reload = []
for module_name in list(sys.modules.keys()):
    if 'prbench' in module_name or 'arm_controller' in module_name:
        modules_to_reload.append(module_name)
        
for module_name in modules_to_reload:
    if module_name in sys.modules:
        del sys.modules[module_name]

# Now import fresh
import prbench
import numpy as np

prbench.register_all_environments()
env = prbench.make(
    "prbench/TidyBot3D-ground-o5-v0",
    render_images=False,
    show_viewer=False,
    show_images=False
)

print("Testing environment with fresh modules...")
obs, info = env.reset(seed=123)
print("Reset successful!")

# Test 3 steps
for i in range(3):
    action = env.action_space.sample() * 0.1  # Small actions
    obs, reward, terminated, truncated, info = env.step(action)
    print(f"Step {i+1} completed successfully")

print("Test completed - Ruckig error resolved!")
env.close()

Testing environment with fresh modules...
Reset successful!



ValueError: -101 is not a valid Result.

In [5]:
# Debug the array slicing issue
import prbench
import numpy as np

prbench.register_all_environments()
env = prbench.make(
    "prbench/TidyBot3D-ground-o5-v0",
    render_images=False,
    show_viewer=False,
    show_images=False
)

# Reset environment
obs, info = env.reset(seed=123)
sim = env.env.env._tidybot_robot_env.sim

# Check DOF allocation
try:
    base_dofs = sim.model._model.body("base_link").jntnum.item()
    print(f"base_dofs: {base_dofs}")
except:
    # Try with robot prefix
    try:
        base_dofs = sim.model._model.body("robot_1_base_link").jntnum.item()
        print(f"base_dofs (with prefix): {base_dofs}")
    except Exception as e:
        print(f"Error finding base_link: {e}")
        # Fallback: examine all body names
        print("\nAll body names:")
        for i in range(sim.model._model.nbody):
            body_name = sim.model._model.body_id2name(i)
            if body_name and ("base" in body_name.lower() or "link" in body_name.lower()):
                jntnum = sim.model._model.body(body_name).jntnum
                print(f"  {body_name}: jntnum = {jntnum}")

arm_dofs = 7
print(f"arm_dofs: {arm_dofs}")

# Check the actual qpos structure
print(f"\nTotal qpos length: {len(sim.data.qpos)}")
print(f"qpos values: {sim.data.qpos[:15]}...")  # First 15 values

# Check what the arm controller thinks its qpos is
arm_ctrl = env.env.env._tidybot_robot_env.arm_controller
print(f"\nArm controller qpos (what it sees): {arm_ctrl.qpos}")
print(f"Retract qpos (what it should be): {arm_ctrl.retract_qpos}")

env.close()



base_dofs: 3
arm_dofs: 7

Total qpos length: 53
qpos values: [ 1.31056776e-17  8.69998172e-17 -1.17386117e-04  1.00000000e+00
 -8.53713322e-16  8.59210925e-16 -9.06030218e-17  7.95454636e-17
  4.28346651e-18 -1.17386117e-04  1.00000000e+00 -4.44085213e-16
 -1.71380155e-15 -2.37468185e-17  1.79760492e-16]...

Arm controller qpos (what it sees): [ 1.00000000e+00 -8.53713322e-16  8.59210925e-16 -9.06030218e-17
  7.95454636e-17  4.28346651e-18 -1.17386117e-04]
Retract qpos (what it should be): [ 0.         -0.34906585  3.14159265 -2.54818071  0.         -0.87266463
  1.57079633]


In [3]:
env.env.env._tidybot_robot_env.sim.model._joint_name2id["joint_1"]

8

In [8]:
env.env.env._tidybot_robot_env.sim.model._tendon_name2id["split"]

0